In [1]:
import json
import os
import time
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
# Funções para consultar DFs
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Consultando códigos de anúncio(ml_code) onde 'logistic_type=fulfillment'

In [3]:
base_url = "https://api.mercadolibre.com/users/233632476/items/search?logistic_type=fulfillment"

params = {
    "limit": 100,
    "offset": 0,
}

headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

json_list = []

try:
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        print(data)
        if "results" in data:
            json_list.extend(data["results"])
            print(data["results"])
        else:
            break

        # Verifique se há mais páginas
        if "paging" in data:
            total_data = data["paging"].get("total")

            total_pages = math.ceil(total_data / params["limit"])
            print(f"Total de páginas a serem processadas: {total_pages}")
            print(f'Offset atual: {params["offset"]}')

            if params["offset"] >= total_pages * params["limit"]:
                break

            params["offset"] += params["limit"]
        else:
            break

except requests.exceptions.RequestException as req_err:
    print(f"Erro ao fazer a requisição para {base_url}: {req_err}")
except Exception as e:
    print(f"Erro não esperado: {e}")

print(f"Total esperado de dados: {total_data}")
print(f"Total de dados coletados: {len(json_list)}")

{'seller_id': '233632476', 'results': ['MLB3778561802', 'MLB3436515963', 'MLB3778538730', 'MLB924922735', 'MLB949771924', 'MLB949776093', 'MLB949788598', 'MLB949790387', 'MLB950297972', 'MLB950303724', 'MLB951043222', 'MLB1363602423', 'MLB1397153967', 'MLB1398114866', 'MLB1399101853', 'MLB1400763583', 'MLB1401964928', 'MLB1401946009', 'MLB1403147752', 'MLB1403024540', 'MLB1403999905', 'MLB1403934211', 'MLB1403920691', 'MLB1403791227', 'MLB1403763907', 'MLB1416821703', 'MLB1418498416', 'MLB1418470905', 'MLB1418466676', 'MLB1417658704', 'MLB1418014629', 'MLB1418532096', 'MLB1425101880', 'MLB1431387563', 'MLB1435969309', 'MLB1435852259', 'MLB1435836214', 'MLB1400355425', 'MLB1440291959', 'MLB1440288810', 'MLB1440247563', 'MLB1417877602', 'MLB1425113746', 'MLB1453957901', 'MLB1453221977', 'MLB1457193551', 'MLB1459840256', 'MLB1459799426', 'MLB1459799420', 'MLB1459771919', 'MLB1461271134', 'MLB1461223727', 'MLB1461203307', 'MLB1461199091', 'MLB1461189878', 'MLB1461717028', 'MLB1461705752', 

### Pesquisando itens onde 'logistic_type=fulfillment', usando o ml_code

In [4]:
json_list_item = []

c = 1
for item in json_list:
    base_url = f"https://api.mercadolibre.com/items/{item}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    t = len(json_list)
    print(item)
    print(f"{c}/{t}")
    c += 1

    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        json_list_item.append(data)
        print(f"Tamanho da nova lista: {len(json_list_item)}/{t}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados para o item {item}: {e}")

    # Se c for um múltiplo de 50, aguarde 1 minuto
    if c % 50 == 0:
        print("Esperando 1 minuto...")
        time.sleep(60)

print(f"Tamanho da lista de itens: {len(json_list_item)}")

MLB3778561802
1/440


Tamanho da nova lista: 1/440
MLB3436515963
2/440
Tamanho da nova lista: 2/440
MLB3778538730
3/440
Tamanho da nova lista: 3/440
MLB924922735
4/440
Tamanho da nova lista: 4/440
MLB949771924
5/440
Tamanho da nova lista: 5/440
MLB949776093
6/440
Tamanho da nova lista: 6/440
MLB949788598
7/440
Tamanho da nova lista: 7/440
MLB949790387
8/440
Tamanho da nova lista: 8/440
MLB950297972
9/440
Tamanho da nova lista: 9/440
MLB950303724
10/440
Tamanho da nova lista: 10/440
MLB951043222
11/440
Tamanho da nova lista: 11/440
MLB1363602423
12/440
Tamanho da nova lista: 12/440
MLB1397153967
13/440
Tamanho da nova lista: 13/440
MLB1398114866
14/440
Tamanho da nova lista: 14/440
MLB1399101853
15/440
Tamanho da nova lista: 15/440
MLB1400763583
16/440
Tamanho da nova lista: 16/440
MLB1401964928
17/440
Tamanho da nova lista: 17/440
MLB1401946009
18/440
Tamanho da nova lista: 18/440
MLB1403147752
19/440
Tamanho da nova lista: 19/440
MLB1403024540
20/440
Tamanho da nova lista: 20/440
MLB1403999905
21/440
Taman

In [5]:
# Salvando em um arquivo
file_path = "../../Data/Output/lista_itens.json"

In [6]:
# Escrever a lista em um arquivo JSON
with open(file_path, "w") as file:
    json.dump(json_list_item, file)

In [ ]:
file_path = "../../Data/Output/lista_itens.json"

with open(file_path, "r") as file:
    json_list_item = json.load(file)

print(f"Tamanho da lista de itens: {len(json_list_item)}")
print(json_list_item)

### Construindo tabela

In [ ]:
df = pd.DataFrame(json_list_item)

print(f"Tamanho do dataframe de itens: {df.shape}")
df.sample()

#### attributes: SELLER_SKU

In [ ]:
resultados_attributes = []

for item in json_list_item:
    # Extrair os valores desejados
    first_id = item["id"]
    inventory_id = item["inventory_id"]
    variations = item["variations"]
    status = item["status"]
    catalog_product_id = item["catalog_product_id"]
    seller_custom_field = item["seller_custom_field"]
    catalog_listing = item["catalog_listing"]
    logistic_type = item["shipping"]["logistic_type"]
    item_relations = item["item_relations"]

    # Procurar em "attributes" onde "id" é "SELLER_SKU"
    seller_sku_entry = next(
        (attr for attr in item["attributes"] if attr["id"] == "SELLER_SKU"), None
    )

    # Obter os valores de "value_name" e "value_id" se a entrada existir, caso contrário, definir como None
    attribute_value_name = seller_sku_entry["value_name"] if seller_sku_entry else None
    attribute_value_id = seller_sku_entry["value_id"] if seller_sku_entry else None

    # attribute_value_name = item["attributes"][0]["value_name"]
    # attribute_value_id = item["attributes"][0]["value_id"]

    # Adicionar os resultados_attributes à lista
    resultados_attributes.append(
        {
            "ml_code": first_id,
            "inventory_id": inventory_id,
            # "logistic_type": logistic_type,
            # "sku": attribute_value_name,
            "status": status,
            "variations": variations,
            # "attribute_value_id": attribute_value_id,
            # "catalog_product_id": catalog_product_id,
            # "seller_custom_field": seller_custom_field,
            "catalog_listing": catalog_listing,
            # "item_relations": item_relations
        }
    )

# Exibir os resultados
print(resultados_attributes)
df_sku = pd.DataFrame(resultados_attributes)
print(df_sku.shape)
df_sku.sample()

#### variations: variation_id,  attribute_combination: value_id, value_name, seller_sku ,inventory_id 

In [ ]:
resultados_variations = []

for item in json_list_item:
    # Extrair os valores comuns para cada item
    first_id = item.get("id")
    inventory_id = item.get("inventory_id")
    logistic_type = item.get("shipping", {}).get("logistic_type")

    # Iterar sobre cada variação no item
    for variacao in item.get("variations", []):
        # Extrair os valores específicos para cada variação
        variation_id = variacao.get("id")
        variation_seller_sku = variacao.get("seller_custom_field")
        variation_inventory_id = variacao.get("inventory_id")
        attribute_combination = variacao.get("attribute_combinations", [{}])[0]
        value_id = attribute_combination.get("value_id")
        value_name = attribute_combination.get("value_name")
        item_relations = attribute_combination.get("item_relations", [{}])[0]

        # Adicionar os resultados_variations à lista
        resultados_variations.append(
            {
                "ml_code": first_id,
                "inventory_id": inventory_id,
                # "logistic_type": logistic_type,
                "variation_id": variation_id,
                # "value_id": value_id,
                "value_name": value_name,
                # "var_seller_sku": variation_seller_sku,
                "variation_inventory_id": variation_inventory_id,
                # "item_relations":item_relations,
            }
        )

# Exibir os resultados_variations
print(resultados_variations)
df_variations = pd.DataFrame(resultados_variations)
print(df_variations.shape)
df_variations.sample()

In [ ]:
df_variations["variation_inventory_id"].value_counts()

In [ ]:
df_variations["inventory_id"].value_counts()

In [ ]:
df_variations["variation_inventory_id"].value_counts()

In [ ]:
id = "UYFJ61970"
xpto = df_variations[df_variations["variation_inventory_id"] == id]
xpto

In [ ]:
id = "MLB2986538160"
xpto = df_variations[df_variations["ml_code"] == id]
xpto

In [ ]:
df_variations["inventory_id"].value_counts()

In [ ]:
df_variations["value_name"].value_counts()

### Unindo as duas tabelas

In [ ]:
# condf(df_sku_var, 'variation_inventory_id', '52181329446')

In [ ]:
df_sku_var = pd.merge(
    df_sku,
    df_variations,
    left_on=["ml_code", "inventory_id"],
    right_on=["ml_code", "inventory_id"],
    how="left",
)
df_sku_var = df_sku_var.drop(["variations"], axis=1)
df_sku_var

#### *if variation_inventory_id = None -> variation_inventory_id == inventory_id && remove inventory_id && variation_inventory_id rename to inventory_id*


In [ ]:
df_sku_var["variation_inventory_id"].fillna(df_sku_var["inventory_id"], inplace=True)

# verificando se união funcionou
result = df_sku_var[df_sku_var["variation_inventory_id"] == df_sku_var["inventory_id"]]
result.sample(2)

In [ ]:
df_sku_var.columns

In [ ]:
# df_sku_var = df_sku_var.drop(['inventory_id'], axis=1)
cols = [
    "ml_code",
    "variation_inventory_id",
    "value_name",
    "variation_id",
    "status",
    "catalog_listing",
]
df_sku_var = df_sku_var[cols]
df_sku_var = df_sku_var.rename(columns={"variation_inventory_id": "inventory_id"})

print(f"Tamanho do dataframe final: {df_sku_var.shape}")
df_sku_var

In [ ]:
df_sku_var["variation_id"] = (
    df_sku_var["variation_id"].astype(str).apply(lambda x: x.rstrip(".0"))
)
df_sku_var["variation_id"] = df_sku_var["variation_id"].replace("nan", np.nan)
df_sku_var

In [ ]:
df_sku_var["variation_id"].value_counts()

In [ ]:
df_sku_var.dtypes

In [ ]:
condf(df_sku_var, "ml_code", "MLB1465231671")

#### verificações

In [ ]:
count_nul = df_sku_var["inventory_id"].isna().sum()
rows_with_nulls = df_sku_var.loc[df_sku_var["inventory_id"].isna()]
print(df_sku_var.shape)
df_sku_var["inventory_id"].value_counts()
count_nul
rows_with_nulls  # fogão cocktop

In [ ]:
# # Mostrar e contar valores únicos em todas as colunas
# for coluna in df_sku_var.columns:
#     unique_values = df_sku_var[coluna].unique()
#     count_unique = df_sku_var[coluna].nunique()
#     print(f"Valores únicos na coluna '{coluna}': {unique_values}")
#     print(f"Número de valores únicos na coluna '{coluna}': {count_unique}")
#     print("\n-----------------------------\n")

### Populando banco de dados

In [ ]:
# df_sku_var.columns

In [ ]:
# df_sku_var.dtypes

In [ ]:
# conn = psycopg2.connect(**db_config)

# cursor = conn.cursor()

# for index, row in df_sku_var.iterrows():
#     insert_query = sql.SQL(
#         "INSERT INTO items (ml_code, inventory_id, value_name, variation_id, status, catalog_listing) VALUES (%s, %s, %s, %s, %s, %s)"
#     )
#     cursor.execute(
#         insert_query,
#         (
#             row["ml_code"],
#             row["inventory_id"],
#             row["value_name"],
#             row["variation_id"],
#             row["status"],
#             row["catalog_listing"],
#         ),
#     )

# conn.commit()

# # Feche o cursor e a conexão
# cursor.close()
# conn.close()
# print("Dados inseridos com sucesso!")

### Buscando itens no banco de dados

In [ ]:
# ler tabela do db
try:
    conn = psycopg2.connect(**db_config)

    query = "SELECT * FROM items;"
    df_items = pd.read_sql(query, conn)
except psycopg2.Error as e:
    print(f"Erro do psycopg2 em 'items': {e}")
    # TODO log
except Exception as e:
    print(f"Erro ao consultar 'items': {e}")
    # TODO log

print(df_items.shape)

In [ ]:
dx = df_items.copy()
dy = df_sku_var.copy()

In [ ]:
## remove linhas de data
dx = dx.drop(columns=["created_at", "updated_at"])

In [ ]:
dx.replace("NaN", np.nan, inplace=True)  # altera de strin para NaN
dx = dx.astype(str)
dy = dy.astype(str)

In [ ]:
dy.sample()

### Atualizar itens no banco de dados

In [ ]:
# Realize a junção (merge) com base nas colunas ml_code e inventory_id
merged_df = pd.merge(
    dy, dx, on=["ml_code", "inventory_id"], how="inner", suffixes=("_sku_var", "_items")
)

# Identifique as linhas com valores diferentes
different_rows = merged_df[
    (merged_df["value_name_sku_var"] != merged_df["value_name_items"])
    | (merged_df["status_sku_var"] != merged_df["status_items"])
    | (merged_df["catalog_listing_sku_var"] != merged_df["catalog_listing_items"])
]

# Exiba as linhas com valores diferentes
different_rows

In [ ]:
# Compare os DataFrames
identicos = dx.equals(dy)

# Exiba o resultado
print("Os DataFrames são idênticos:", identicos)

In [ ]:
# diferencas = df_db.compare(df_new)
diferencas = dx.compare(dy)

# Exiba as diferenças
print("Diferenças entre os DataFrames:")
diferencas

In [ ]:
# Encontrar diferenças usando merge
diferencas = (
    pd.merge(dx, dy, how="outer", indicator=True)
    .query('_merge == "left_only"')
    .drop("_merge", axis=1)
)

# Criar um novo DataFrame apenas com as colunas modificadas
df_atualizado = dx.copy()
df_atualizado[diferencas.columns] = diferencas

# Exibir DataFrame atualizado
print("Novo DataFrame Atualizado:")
df_atualizado

In [ ]:
# Remover linhas onde todos os valores em TODAS as colunas são NaN
df_atualizado_sem_nan = df_atualizado.dropna(how="all", subset=df_atualizado.columns)

# Exibir DataFrame atualizado
print("DataFrame Atualizado sem Linhas com Todos os Valores em Todas as Colunas NaN:")
df_atualizado_sem_nan

In [ ]:
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()


# Iterar sobre as linhas do DataFrame e executar as atualizações no banco de dados
for index, row in df_atualizado_sem_nan.iterrows():
    ml_code = row["ml_code"]
    inventory_id = row["inventory_id"]
    value_name = row["value_name"]
    variation_id = row["variation_id"]
    status = row["status"]
    catalog_listing = row["catalog_listing"]
    updated_at = datetime.now()  # Use a data/hora atual

    # Construir a instrução SQL de atualização
    update_query = sql.SQL(
        "UPDATE items SET value_name = %s, variation_id = %s, status = %s, catalog_listing = %s, updated_at = %s WHERE ml_code = %s AND inventory_id = %s"
    )

    # Executar a instrução SQL
    cursor.execute(
        update_query,
        (
            value_name,
            variation_id,
            status,
            catalog_listing,
            updated_at,
            ml_code,
            inventory_id,
        ),
    )

conn.commit()

cursor.close()
conn.close()

print("Dados inseridos com sucesso!")

### Popule com novos itens

In [ ]:
data = {
    "ml_code": ["MLBXXXXX"],
    "inventory_id": ["xxxxx"],
    "value_name": ["xxxx"],
    "variation_id": ["xxxx"],
    "status": ["xxxx"],
    "catalog_listing": [False],
}

df_ficticio = pd.DataFrame(data)

In [ ]:
# Encontrar linhas onde os pares ml_code e inventory_id em df_ficticio são diferentes de dx
diferenca = pd.merge(
    dx, df_ficticio, on=["ml_code", "inventory_id"], how="right", indicator=True
)

# Filtrar apenas as linhas em que df_ficticio tem valores diferentes de dx
diferenca = diferenca.query('_merge == "right_only"').drop(columns="_merge")
diferenca

In [ ]:
# Selecionar colunas específicas e renomear
diferenca = diferenca[
    [
        "ml_code",
        "inventory_id",
        "value_name_y",
        "variation_id_y",
        "status_y",
        "catalog_listing_y",
    ]
]
diferenca = diferenca.rename(
    columns={
        "value_name_y": "value_name",
        "status_y": "status",
        "catalog_listing_y": "catalog_listing",
    }
)
diferenca

In [ ]:
# # Encontrar linhas onde os pares ml_code e inventory_id em dy são diferentes de dx
# linhas_diferentes = pd.merge(dx, df_ficticio, on=['ml_code', 'inventory_id'], how='right', indicator=True).query('_merge == "right_only"').drop('_merge', axis=1)

# # Exibir as linhas diferentes
# print("Linhas onde ml_code e inventory_id em dy são diferentes de dx:")
# linhas_diferentes

In [ ]:
# # Encontrar linhas que existem em dy, mas não em dx
# linhas_novas = pd.merge(dx, df_ficticio,on=['ml_code', 'inventory_id'], how='right', indicator=True).query('_merge == "right_only"').drop('_merge', axis=1)

# # Exibir as linhas novas
# print("Linhas que existem em dy, mas não em dx:")
# linhas_novas

In [ ]:
# Inserir novos dados no banco de dados
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in diferenca.iterrows():
    insert_query = sql.SQL(
        "INSERT INTO items (ml_code, inventory_id, value_name, status, catalog_listing) VALUES (%s, %s, %s, %s, %s)"
    )
    cursor.execute(
        insert_query,
        (
            row["ml_code"],
            row["inventory_id"],
            row["value_name"],
            row["status"],
            row["catalog_listing"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")